In [1]:
import pandas as pd
tobacco = pd.read_excel("../data/tobacco-retailers.xlsx")

In [2]:
tobacco.head()

,Address Building,Address Street Name,Address City,Address State,Address ZIP,Address Borough
0,740,GERARD AVE,BRONX,NY,10451,Bronx
1,268,WHITE PLAINS RD,BRONX,NY,10473,Bronx
2,73,FEATHERBED LN,BRONX,NY,10452,Bronx
3,330,E GUN HILL RD,BRONX,NY,10467,Bronx
4,456,E 149TH ST,BRONX,NY,10455,Bronx


In [3]:
tobacco.isnull().sum()

Address Building       214
Address Street Name      0
Address City             1
Address State           13
Address ZIP              0
Address Borough          1
dtype: int64

Some of these clearly can't be geolocated. `7 AVENUE` in Manhattan for example is impossible to put an exact position on.

In [4]:
tobacco[tobacco['Address Building'].isnull()]

,Address Building,Address Street Name,Address City,Address State,Address ZIP,Address Borough
299,NaN,EAST 208 STREET,BRONX,NY,10467,Bronx
664,NaN,HUGH J GRANT CIRCLE,BRONX,NY,10462,Bronx
1162,NaN,EAST 167 STREET,BRONX,NY,10452,Bronx
1471,NaN,WESTCHESTER SQ,BRONX,NY,10461,Bronx
1910,NaN,COURT STREET,BROOKLYN,NY,11201,Brooklyn
2043,NaN,CALLAHAN-KELLY PLAYGROUND,BROOKLYN,NY,11233,Brooklyn
2298,NaN,MONTAGUE STREET,BROOKLYN,NY,11201,Brooklyn
2361,NaN,LORIMER ST,BROOKLYN,NY,11211,Brooklyn
3762,NaN,FLATBUSH AVENUE,BROOKLYN,NY,11217,Brooklyn
4036,NaN,13 AVENUE,BROOKLYN,NY,11219,Brooklyn


JFK is a particular pain point, but the addresses which do have coverage should be sufficient.

In [26]:
tobacco[tobacco['Address Street Name'].map(lambda a: 'JFK' in str(a))]

,Address Building,Address Street Name,Address City,Address State,Address ZIP,Address Borough
6716,NaN,JFK AIRPORT,JAMAICA,NY,11430,Queens
6724,NaN,JFK AIRPORT,JAMAICA,NY,11430,Queens
6726,1,JFK INTL AIRPORT BLVD,JAMAICA,NY,11430,Queens
6944,NaN,JFK AIRPORT,JAMAICA,NY,11430,Queens
7067,1,JFK INTL AIRPORT BLVD,JAMAICA,NY,11430,Queens
7212,NaN,JFK AIRPORT,NaN,NY,11430,Queens
7342,1,JFK INTL AIRPORT BLVD,JAMAICA,NY,11430,Queens
7410,NaN,JFK AIRPORT,JAMAICA,NY,11430,Queens
7558,NaN,JFK AIRPORT,JAMAICA,NY,11430,Queens
7563,NaN,JFK AIRPORT,JAMAICA,NY,11430,Queens


Note: `api` is a local copy of my [nyc-geoclient](https://github.com/ResidentMario/nyc-geoclient/blob/master/nyc_geoclient/api.py) fork that works with Python 3. It didn't work via `pip install -e .` unfortunately, so I backed up to just copy-pasting the file in. Sorry.

In [11]:
import sys
sys.path.append('..')
import api

In [5]:
import json

with open("../data/nyc-geocoder-api-key.json") as f:
    keychain = json.loads(f.read())

In [6]:
keychain

{'id': 'e8641bad', 'key': '22c53431c17717ffa7620fa546dd1272'}

In [13]:
geoclient = api.Geoclient(keychain['id'], keychain['key'])

In [14]:
geoclient.address_zip(740, 'GERARD AVE', 10451)

{'assemblyDistrict': '84',
 'bbl': '2024730043',
 'bblBoroughCode': '2',
 'bblTaxBlock': '02473',
 'bblTaxLot': '0043',
 'bikeLane': '2',
 'boardOfElectionsPreferredLgc': '1',
 'boePreferredStreetName': 'GERARD AVENUE',
 'boePreferredstreetCode': '23502001',
 'buildingIdentificationNumber': '2002897',
 'censusBlock2000': '2001',
 'censusBlock2010': '6001',
 'censusTract1990': '  5901',
 'censusTract2000': '  5901',
 'censusTract2010': '  63  ',
 'cityCouncilDistrict': '08',
 'civilCourtDistrict': '02',
 'coincidentSegmentCount': '1',
 'communityDistrict': '204',
 'communityDistrictBoroughCode': '2',
 'communityDistrictNumber': '04',
 'communitySchoolDistrict': '07',
 'condominiumBillingBbl': '0000000000',
 'congressionalDistrict': '15',
 'cooperativeIdNumber': '0000',
 'crossStreetNamesFlagIn': 'E',
 'dcpPreferredLgc': '01',
 'dotStreetLightContractorArea': '2',
 'dynamicBlock': '223',
 'electionDistrict': '016',
 'fireBattalion': '17',
 'fireCompanyNumber': '068',
 'fireCompanyType': 

In [19]:
geoclient.place('GRAND CENTRAL TERMINAL', 'Manhattan')

{'assemblyDistrict': '73',
 'bbl': '1012800001',
 'bblBoroughCode': '1',
 'bblTaxBlock': '01280',
 'bblTaxLot': '0001',
 'boardOfElectionsPreferredLgc': '1',
 'boePreferredStreetName': 'GRAND CENTRAL TERMINAL',
 'boePreferredstreetCode': '12168501',
 'boroughCode1In': '1',
 'buildingIdentificationNumber': '1035381',
 'censusBlock2000': '3006',
 'censusBlock2010': '1007',
 'censusTract1990': '  92  ',
 'censusTract2000': '  92  ',
 'censusTract2010': '  92  ',
 'cityCouncilDistrict': '04',
 'civilCourtDistrict': '09',
 'coincidentSegmentCount': '1',
 'communityDistrict': '105',
 'communityDistrictBoroughCode': '1',
 'communityDistrictNumber': '05',
 'communitySchoolDistrict': '02',
 'condominiumBillingBbl': '0000000000',
 'congressionalDistrict': '12',
 'cornerCode': 'NE',
 'crossStreetNamesFlagIn': 'E',
 'dcpCommercialStudyArea': '11031',
 'dcpPreferredLgc': '01',
 'dotStreetLightContractorArea': '1',
 'dynamicBlock': '301',
 'electionDistrict': '008',
 'fireBattalion': '08',
 'fireCom

I particularly like the playground which appears to smell smokes, too bad the true address for this entry can't be recovered. This likely is an error somewhere in the system, perhaps they meant to bodega across the street.

In [20]:
geoclient.place('CALLAHAN-KELLY PLAYGROUND', 'Manhattan')

{'boroughCode1In': '1',
 'crossStreetNamesFlagIn': 'E',
 'firstBoroughName': 'MANHATTAN',
 'firstStreetNameNormalized': 'CALLAHAN-KELLY PLAYGROUND',
 'geosupportFunctionCode': '1B',
 'geosupportReturnCode': 'EE',
 'geosupportReturnCode2': 'EE',
 'message': "'CALLAHAN-KELLY PLAYGROUND' NOT RECOGNIZED. THERE ARE 002 SIMILAR NAMES.",
 'message2': "'CALLAHAN-KELLY PLAYGROUND' NOT RECOGNIZED. THERE ARE 002 SIMILAR NAMES.",
 'numberOfStreetCodesAndNamesInList': '02',
 'reasonCode': '2',
 'reasonCode1a': '2',
 'reasonCode1e': '2',
 'reasonCode2': '2',
 'returnCode1a': 'EE',
 'returnCode1e': 'EE',
 'streetCode1': '11204501',
 'streetCode2': '14529002',
 'streetName1': 'CALLENDER PG',
 'streetName1In': 'CALLAHAN-KELLY PLAYGROUND',
 'streetName2': 'CALVIN PLACE',
 'workAreaFormatIndicatorIn': 'C'}

In [72]:
from tqdm import tqdm_notebook

lats = []
longs = []
failures = []
failure_outputs = []
for row in tqdm_notebook(tobacco.iterrows()):
    srs = row[1]
    address_modified = ''.join(i for i in str(srs['Address Building']) if i.isdigit())
    if address_modified:
        geo = geoclient.address_zip(address_modified, srs['Address Street Name'], srs['Address ZIP'])
        try:
            lats.append(geo['latitude'])
            longs.append(geo['longitude'])
        except KeyError as e:
            lats.append(None)
            longs.append(None)
            failures.append(srs)
            failure_outputs.append(geo)
    elif srs['Address Borough']:
        geo = geoclient.place(srs['Address Street Name'], srs['Address Borough'])
        try:
            lats.append(geo['latitude'])
            longs.append(geo['longitude'])
        except KeyError:
            lats.append(None)
            longs.append(None)
            failures.append(srs)
            failure_outputs.append(geo)
    else:
        lats.append(None)
        longs.append(None)
        failures.append(srs)
        failure_outputs.append(None)

In [73]:
len(failures)

306

In [75]:
tobacco['Latitude'] = lats
tobacco['Longitude'] = longs
tobacco.head()

,Address Building,Address Street Name,Address City,Address State,Address ZIP,Address Borough,Latitude,Longitude
0,740,GERARD AVE,BRONX,NY,10451,Bronx,40.824546,-73.926663
1,268,WHITE PLAINS RD,BRONX,NY,10473,Bronx,40.810945,-73.855668
2,73,FEATHERBED LN,BRONX,NY,10452,Bronx,40.846355,-73.917433
3,330,E GUN HILL RD,BRONX,NY,10467,Bronx,40.878826,-73.873294
4,456,E 149TH ST,BRONX,NY,10455,Bronx,40.815325,-73.915900


### Errors encountered

The geocoder doesn't like having characters in its address input. Strangely, if there is a valid character indicating a building subrange, the geocoder will return different output from what you'd get if you passed it the "parent" address without the character. For example, consider the difference between the following two outputs, one for `719A LYDIG AVE` and one for `719 LYDIG AVE`:

In [49]:
geoclient.address_zip('719A', 'LYDIG AVE', 10462)

{'bbl': '2043180057',
 'bblBoroughCode': '2',
 'bblTaxBlock': '04318',
 'bblTaxLot': '0057',
 'buildingIdentificationNumber': '2049402',
 'condominiumBillingBbl': '0000000000',
 'cooperativeIdNumber': '0000',
 'crossStreetNamesFlagIn': 'E',
 'dcpCommercialStudyArea': '23071',
 'firstBoroughName': 'BRONX',
 'firstStreetCode': '24752001010',
 'firstStreetNameNormalized': 'LYDIG AVENUE',
 'geosupportFunctionCode': '1B',
 'geosupportReturnCode': '42',
 'geosupportReturnCode2': '00',
 'gi5DigitStreetCode1': '47520',
 'gi5DigitStreetCode2': '22920',
 'gi5DigitStreetCode3': '47520',
 'giBoroughCode1': '2',
 'giBoroughCode2': '2',
 'giBoroughCode3': '2',
 'giBuildingIdentificationNumber1': '2049402',
 'giBuildingIdentificationNumber2': '2049402',
 'giBuildingIdentificationNumber3': '2049402',
 'giDcpPreferredLgc1': '01',
 'giDcpPreferredLgc2': '01',
 'giDcpPreferredLgc3': '01',
 'giHighHouseNumber1': '719A',
 'giHighHouseNumber2': '2147',
 'giHighHouseNumber3': '719',
 'giLowHouseNumber1': '71

In [41]:
geoclient.address_zip('719', 'LYDIG AVE', 10462)

{'assemblyDistrict': '80',
 'bbl': '2043180057',
 'bblBoroughCode': '2',
 'bblTaxBlock': '04318',
 'bblTaxLot': '0057',
 'boardOfElectionsPreferredLgc': '1',
 'boePreferredStreetName': 'LYDIG AVENUE',
 'boePreferredstreetCode': '24752001',
 'buildingIdentificationNumber': '2049402',
 'censusBlock2000': '4000',
 'censusBlock2010': '2001',
 'censusTract1990': ' 22402',
 'censusTract2000': ' 22402',
 'censusTract2010': ' 22403',
 'cityCouncilDistrict': '13',
 'civilCourtDistrict': '01',
 'coincidentSegmentCount': '1',
 'communityDistrict': '211',
 'communityDistrictBoroughCode': '2',
 'communityDistrictNumber': '11',
 'communitySchoolDistrict': '11',
 'condominiumBillingBbl': '0000000000',
 'congressionalDistrict': '14',
 'cooperativeIdNumber': '0000',
 'crossStreetNamesFlagIn': 'E',
 'dcpCommercialStudyArea': '23071',
 'dcpPreferredLgc': '01',
 'dotStreetLightContractorArea': '2',
 'dynamicBlock': '403',
 'electionDistrict': '023',
 'fireBattalion': '20',
 'fireCompanyNumber': '041',
 'f

In other cases it simply can't geocode entries provided in the dataset at all. For example, here is what it has to say about `159 EINSTEIN LOOP` (which [does appear on Google Maps](https://www.google.com/maps/place/159+Einstein+Loop+N,+Bronx,+NY+10475/@40.8653756,-73.8248539,16.75z/data=!4m5!3m4!1s0x89c28cbc2647680f:0x793242db657050d9!8m2!3d40.865348!4d-73.8226292)):

In [57]:
geoclient.address_zip('159', 'EINSTEIN LOOP', 10475)

{'crossStreetNamesFlagIn': 'E',
 'firstBoroughName': 'BRONX',
 'firstStreetCode': '23037001010',
 'firstStreetNameNormalized': 'EINSTEIN LOOP',
 'geosupportFunctionCode': '1B',
 'geosupportReturnCode': '42',
 'geosupportReturnCode2': '42',
 'houseNumber': '159',
 'houseNumberIn': '159',
 'houseNumberSortFormat': '000159000AA',
 'message': 'ADDRESS NUMBER OUT OF RANGE',
 'message2': 'ADDRESS NUMBER OUT OF RANGE',
 'returnCode1a': '42',
 'returnCode1e': '42',
 'streetName1In': 'EINSTEIN LOOP',
 'workAreaFormatIndicatorIn': 'C',
 'zipCodeIn': '10475'}

The fix was to include these in error output, for inspection later.

Fixing error number one above resulted in some addresses consisting only of letters getting mapped to addresses consisting only of an empty string, which the geocoder won't accept. The fix: these were pre-checked beforehand and sent to the errors bin as well if caught.

We'll give things that errored out to the Google geocoder.

In [76]:
import geocoder
g = geocoder.google('Mountain View, CA')

In [81]:
corrected_lat_longs = []

for srs in tqdm_notebook(failures):
    if not srs['Address Building']:
        add = ""
    else:
        add = srs['Address Building']
    if not srs['Address Borough']:
        b = ""
    else:
        b = srs['Address Borough']
    q = str(add) + " " + str(srs['Address Street Name']) + ", " + str(b) + ", NY"
    g = geocoder.google(q)
    try:
        lat, long = g.latlng
        corrected_lat_longs.append((lat, long))
    except:
        corrected_lat_longs.append(None)

In [82]:
corrected_lat_longs

[(40.8653356, -73.8226384),
 (40.8589734, -73.8862766),
 (40.8370495, -73.86542949999999),
 (40.8336112, -73.8608517),
 (40.8701903, -73.8855179),
 (40.84149439999999, -73.8432031),
 (40.8370495, -73.86542949999999),
 (40.8223985, -73.8109607),
 (40.80724970000001, -73.85760909999999),
 (40.84110700000001, -73.8428263),
 (40.680315, -73.996887),
 (40.6947901, -73.99429529999999),
 (40.6947901, -73.99429529999999),
 (40.71477300000001, -73.94950419999999),
 (40.6940175, -73.93749509999999),
 (40.680315, -73.996887),
 (40.67921, -73.9280339),
 (40.693147, -73.99508),
 (40.60316, -73.981679),
 (40.67684999999999, -73.896755),
 (40.6295097, -73.9441461),
 (40.628263, -73.999149),
 (40.6781784, -73.9441579),
 (40.67921, -73.9280339),
 (40.7668893, -73.97907099999999),
 (40.7830603, -73.9712488),
 (40.7830603, -73.9712488),
 (40.7830603, -73.9712488),
 (40.7830603, -73.9712488),
 (40.7830603, -73.9712488),
 (40.759011, -73.9844722),
 (40.7947292, -73.971598),
 (40.7830603, -73.9712488),
 (40

In [89]:
i = 0
lats_c = lats.copy()
for ind, l in enumerate(lats_c):
    if not l:
        try:
            lats_c[ind] = corrected_lat_longs[i][0]
        except:
            pass
        i += 1

In [90]:
lats_c

[40.82454587497495,
 40.81094464094616,
 40.84635460806038,
 40.87882555133326,
 40.815324504874766,
 40.85448928073022,
 40.83017793850434,
 40.87991822710596,
 40.86553104563283,
 40.82525989029229,
 40.8734976155627,
 40.91002566220964,
 40.840485733130556,
 40.873669890657986,
 40.86212224936796,
 40.895213472053136,
 40.85415806858476,
 40.8444980324205,
 40.874993806522646,
 40.905289053109875,
 40.83246340365935,
 40.869348413260475,
 40.819259632602765,
 40.83603253046173,
 40.908203874844446,
 40.81308183278152,
 40.82708686089327,
 40.843812817095056,
 40.86260710520571,
 40.848496123508994,
 40.84811343418139,
 40.87507992803471,
 40.84457181842375,
 40.84698421912153,
 40.857825047594,
 40.87590094196857,
 40.83074608314149,
 40.82893751701337,
 40.85839338377227,
 40.822669916574895,
 40.84201873692707,
 40.81882865112807,
 40.84886199603753,
 40.85566823284615,
 40.88950268722485,
 40.82392125994305,
 40.828337902466785,
 40.842051272131286,
 40.858124734816435,
 40.84034

In [92]:
i = 0
longs_c = longs.copy()
for ind, l in enumerate(longs_c):
    if not l:
        try:
            longs_c[ind] = corrected_lat_longs[i][1]
        except:
            pass
        i += 1

In [98]:
longs_c

[-73.9266628282542,
 -73.85566755931745,
 -73.91743293791271,
 -73.87329360964058,
 -73.91589990977934,
 -73.86684773138523,
 -73.91456581793403,
 -73.87608686147225,
 -73.86202794275785,
 -73.8784107921435,
 -73.85428068870672,
 -73.89661268706902,
 -73.85541192209944,
 -73.88534068185191,
 -73.90922121571127,
 -73.86084142944249,
 -73.84351114423205,
 -73.88147307436307,
 -73.90123409829017,
 -73.89740145583578,
 -73.82777779785181,
 -73.82519832096376,
 -73.84807323139738,
 -73.92250510650176,
 -73.90374611199744,
 -73.90838857411939,
 -73.91761222051879,
 -73.91342788919597,
 -73.89820843476994,
 -73.91092055654339,
 -73.89335828867883,
 -73.90917821318018,
 -73.84630947437739,
 -73.91512976923352,
 -73.8856463541947,
 -73.87957293672517,
 -73.89168053308425,
 -73.88900591576031,
 -73.89752441067942,
 -73.91664576472935,
 -73.91108102453146,
 -73.91852205482652,
 -73.88405329898758,
 -73.9010567471798,
 -73.89858679233694,
 -73.8991528403272,
 -73.8821956360572,
 -73.91417469746112

In [96]:
tobacco['Latitude'] = lats_c
tobacco['Longitude'] = longs_c

In [100]:
tobacco[tobacco['Latitude'].isnull()]

,Address Building,Address Street Name,Address City,Address State,Address ZIP,Address Borough,Latitude,Longitude
6330,NaN,EAST 86 STREET,NEW YORK,NY,10028,Manhattan,NaN,NaN
7720,1,JFK INTL AIRPORT BLVD,JAMAICA,NY,11430,Queens,NaN,NaN


In [101]:
tobacco[tobacco['Longitude'].isnull()]

,Address Building,Address Street Name,Address City,Address State,Address ZIP,Address Borough,Latitude,Longitude
6330,NaN,EAST 86 STREET,NEW YORK,NY,10028,Manhattan,NaN,NaN
7720,1,JFK INTL AIRPORT BLVD,JAMAICA,NY,11430,Queens,NaN,NaN


In [102]:
tobacco = tobacco[tobacco['Latitude'].notnull()]

In [104]:
tobacco.to_csv("../data/tobacco-retailers-geocoded.csv", encoding='utf-8')